In [1]:
from atlassian import Jira
from bidict import bidict

# jira_info = {
#     'url': 'https://tangramcare.atlassian.net/',
#     'username': 'xxx@ownedoutcomes.com',
#     'api_token': 'xxxxxxxxxxxxxxxx',
# }

jira_info = {
    'url': 'https://tangramcare.atlassian.net/',
    'username': 'awaz@ownedoutcomes.com',
    'api_token': '6Pu9J7zSN4wBqwqgSREsAE08',
}



jira = Jira(
            url=jira_info['url'],
            username=jira_info['username'],
            password=jira_info['api_token'],
            cloud=True)

In [42]:
import datetime
def datetime_to_date_converter(date):
    from datetime import datetime
    from dateutil import parser
    if isinstance(date, datetime):
        return date.strftime("%Y-%m-%d")
    return parser.parse(date).strftime("%Y-%m-%d")


def get_schema_from_sheet(sheet, type): # TODO schema validation
    schema = {}

    if type == 'FE':
        schema['offset_row'] = 1 # offset of WBS: TODO search for WBS
        schema['offset_col'] = 0 # offset of WBS: TODO search for WBS
        schema['columns'] = { # TODO make it dynamic
            'Category': 'A',
            'WBS': 'C',
            'Name': 'B',
            'Description': 'M',
            'Depends on': 'D',
            'Estimate [h]': 'E',
            'Jira ID': 'V',
            'Start time': 'S', # Planned and actual
            'Finish time': 'T', # Planned and actual
            # 'Duration [h]': 'L'
        }
        schema['data_offset_row'] = 3
    
    if type == 'BE' or type == 'QA':
        schema['offset_row'] = 1 # offset of WBS: TODO search for WBS
        schema['offset_col'] = 0 # offset of WBS: TODO search for WBS
        schema['columns'] = { # TODO make it dynamic
            'WBS': 'A',
            'Name': 'B',
            'Description': 'C',
            'Depends on': 'D',
            'Estimate [h]': 'H',
            'Jira ID': 'O',
            'Start time': 'J', # Planned and actual
            'Finish time': 'K', # Planned and actual
            'Duration [h]': 'L',
            'Category': 'S',
            'Sprint': 'Q'
        }
        schema['data_offset_row'] = 3

    return schema
    

def wbs_regex_check(string, separator = '.'):
    import re
    return True if re.match('\A([0-9]+\\' + separator + '?)+\Z', string) else False


def wbs_regex_task(string):
    import re
    return True if re.match('\A[0-9]+\Z', string) else False


def check_wbs_level(wbs, separator = '.'):
    return len(wbs.split(separator))


def get_wbs_ancestor(wbs, separator = '.'):
    if len(wbs.split(wbs)) == 1:
        return wbs.split(wbs)[0]
    else:
        return '.'.join(wbs.split('.')[0:-1])


def check_if_successor(wbs, successor):
    wbs_split = wbs.split('.')
    successor_split = successor.split('.')
    if len(wbs_split) >= len(successor_split):
        return False
    for i in range(len(wbs_split)):
        if wbs_split[i] != successor_split[i]:
            return False
    return True


def get_wbs_lowest_level_task_number(wbs, separator = '.'):
    return wbs.split(separator)[-1]


def expand_wbs_range(start, finish, separator = '.'):
    if check_wbs_level(start, separator) != check_wbs_level(finish, separator):
        print('ERROR: WBS range level differs: ', start, ' - ', finish)
        return []
    if get_wbs_ancestor(start, separator) != get_wbs_ancestor(finish, separator):
        print('ERROR: WBS range ancestor differs: ', start, ' - ', finish)
        return []
    lowest_level_task = list(range(int(get_wbs_lowest_level_task_number(start, separator)), 1 + int(get_wbs_lowest_level_task_number(finish, separator))))
    wbs_tasks = []
    for i in lowest_level_task:
        if get_wbs_ancestor(start, separator) == '':
            wbs_tasks.append(str(i))
        else:
            wbs_tasks.append(get_wbs_ancestor(start, separator) + '.' + str(i))
    return wbs_tasks
    

def parse_dependencies(dependency_str, wbs_separator = '.'):
    wbs_tasks = []
    for i in str(dependency_str).split(','):
        if wbs_regex_check(i.strip(), wbs_separator):
            wbs_tasks.append(i)
        else:
            range = i.split('-')
            if len(range) != 2:
                print("ERROR: It is not a range: ", range)
                continue
            if wbs_regex_check(range[0].strip(), wbs_separator) == False or wbs_regex_check(range[1].strip(), wbs_separator) == False:
                print("ERROR: It is not a range: ", range)
                continue
            wbs_tasks = wbs_tasks + expand_wbs_range(range[0].strip(), range[1].strip(), wbs_separator)
    return wbs_tasks


def get_value_for_wbs(schema, sheets, wbs, column_name):
    for sheet in sheets:
        i = schema['data_offset_row']
        none_count = 0
        while none_count < 50:
            if sheet[schema['columns']['WBS'] + str(i)].value is not None:
                if wbs == str(sheet[schema['columns']['WBS'] + str(i)].value):
                    return str(sheet[schema['columns'][column_name] + str(i)].value)

                none_count = 0
            else:
                none_count = none_count + 1
            i = i + 1 
    return None


def get_jira_key_for_wbs(schema, sheets, wbs):
    return get_value_for_wbs(schema, sheets, wbs, 'Jira ID')


def get_start_finish_from_child(sheets, schema, wbs):
    from dateutil import parser
    min = None
    max = None
    for sheet in sheets:
        i = schema['data_offset_row']
        none_count = 0
        while none_count < 50:
            if sheet[schema['columns']['WBS'] + str(i)].value is not None:

                if check_if_successor(wbs, str(sheet[schema['columns']['WBS'] + str(i)].value)):

                    if min is None and sheet[schema['columns']['Start time'] + str(i)].value is not None:
                        if isinstance(sheet[schema['columns']['Start time'] + str(i)].value, datetime.datetime):
                            min = sheet[schema['columns']['Start time'] + str(i)].value
                        else:
                            min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)

                    if max is None and sheet[schema['columns']['Finish time'] + str(i)].value is not None:
                        if isinstance(sheet[schema['columns']['Finish time'] + str(i)].value, datetime.datetime):
                            max = sheet[schema['columns']['Finish time'] + str(i)].value
                        else:
                            max = parser.parse(sheet[schema['columns']['Finish time'] + str(i)].value)

                    if sheet[schema['columns']['Start time'] + str(i)].value is not None:
                        if isinstance(sheet[schema['columns']['Start time'] + str(i)].value, datetime.datetime):
                            if sheet[schema['columns']['Start time'] + str(i)].value < min:
                                min = sheet[schema['columns']['Start time'] + str(i)].value
                        else:
                            if parser.parse(sheet[schema['columns']['Start time'] + str(i)].value) < min:
                                min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)

                    if sheet[schema['columns']['Finish time'] + str(i)].value is not None: 
                        if isinstance(sheet[schema['columns']['Finish time'] + str(i)].value, datetime.datetime):
                            if sheet[schema['columns']['Finish time'] + str(i)].value > max:
                                max = sheet[schema['columns']['Finish time'] + str(i)].value
                        else:
                            if parser.parse(sheet[schema['columns']['Finish time'] + str(i)].value) > max:
                                max = parser.parse(sheet[schema['columns']['Finish time'] + str(i)].value)

                none_count = 0
            else:
                none_count = none_count + 1

            i = i + 1 
    return min, max


def get_min_from_start(sheets, schema):
    from dateutil import parser
    min = None
    for sheet in sheets:
        i = schema['data_offset_row']
        none_count = 0
        while none_count < 50:
            if sheet[schema['columns']['WBS'] + str(i)].value is not None:
                if min is None and sheet[schema['columns']['Start time'] + str(i)].value is not None:
                    if isinstance(sheet[schema['columns']['Start time'] + str(i)].value, datetime.datetime):
                        min = sheet[schema['columns']['Start time'] + str(i)].value
                    else:
                        min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)

                if sheet[schema['columns']['Start time'] + str(i)].value is not None: 
                    if isinstance(sheet[schema['columns']['Start time'] + str(i)].value, datetime.datetime):
                        if sheet[schema['columns']['Start time'] + str(i)].value < min:
                            min = sheet[schema['columns']['Start time'] + str(i)].value
                    else:
                        if parser.parse(sheet[schema['columns']['Start time'] + str(i)].value) < min:
                            min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)

                none_count = 0
            else:
                none_count = none_count + 1
            i = i + 1 
    return min


def schema_BE(sheets, sheet, schema, index, task_json):
    ## actual start and finish time
    min, max = get_start_finish_from_child(sheets, schema, sheet[schema['columns']['WBS'] + str(index)].value)

    if min is not None:
        task_json['customfield_11701'] = min.strftime("%Y-%m-%d")
        sheet[schema['columns']['Start time'] + str(index)].value = min
    if max is not None:
        task_json['customfield_11702'] = max.strftime("%Y-%m-%d")
        sheet[schema['columns']['Finish time'] + str(index)].value = max

    if sheet[schema['columns']['Start time'] + str(index)].value is not None and min is None:
        task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
    if sheet[schema['columns']['Finish time'] + str(index)].value is not None and max is None:
        task_json['customfield_11702'] = datetime_to_date_converter(sheet[schema['columns']['Finish time'] + str(index)].value)

    if 'customfield_11701' not in task_json and sheet[schema['columns']['Sprint'] + str(index)].value is not None:
        if sheet[schema['columns']['Sprint'] + str(index)].value == 'Sprint 5':
            sheet[schema['columns']['Start time'] + str(index)].value = datetime.datetime(year=2021, month=4, day=30)
            task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
        if sheet[schema['columns']['Sprint'] + str(index)].value == 'Sprint 6':
            sheet[schema['columns']['Start time'] + str(index)].value = datetime.datetime(year=2021, month=5, day=14)
            task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
        if sheet[schema['columns']['Sprint'] + str(index)].value == 'Sprint 7':
            sheet[schema['columns']['Start time'] + str(index)].value = datetime.datetime(year=2021, month=5, day=28)
            task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
        if sheet[schema['columns']['Sprint'] + str(index)].value == 'Sprint 8':
            sheet[schema['columns']['Start time'] + str(index)].value = datetime.datetime(year=2021, month=6, day=11)
            task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
        if sheet[schema['columns']['Sprint'] + str(index)].value == 'Sprint 9':
            sheet[schema['columns']['Start time'] + str(index)].value = datetime.datetime(year=2021, month=6, day=25)
            task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
        if sheet[schema['columns']['Sprint'] + str(index)].value == 'Sprint 10':
            sheet[schema['columns']['Start time'] + str(index)].value = datetime.datetime(year=2021, month=6, day=29)
            task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
        if sheet[schema['columns']['Sprint'] + str(index)].value == 'Sprint 11':
            sheet[schema['columns']['Start time'] + str(index)].value = datetime.datetime(year=2021, month=7, day=13)
            task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
        if sheet[schema['columns']['Sprint'] + str(index)].value == 'Sprint 12':
            sheet[schema['columns']['Start time'] + str(index)].value = datetime.datetime(year=2021, month=7, day=27)
            task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)

        if 'customfield_11701' in task_json:
            if sheet[schema['columns']['Estimate [h]'] + str(index)].value is not None:
                sheet[schema['columns']['Finish time'] + str(index)].value = sheet[schema['columns']['Start time'] + str(index)].value + datetime.timedelta(hours = float(sheet[schema['columns']['Estimate [h]'] + str(index)].value)*3)
                task_json['customfield_11702'] = datetime_to_date_converter(sheet[schema['columns']['Finish time'] + str(index)].value)
            else:
                sheet[schema['columns']['Finish time'] + str(index)].value = sheet[schema['columns']['Start time'] + str(index)].value + datetime.timedelta(days = 1)
                task_json['customfield_11702'] =  datetime_to_date_converter(sheet[schema['columns']['Finish time'] + str(index)].value)



    global_min = get_min_from_start(sheets, schema)
    if 'customfield_11701' not in task_json:
        task_json['customfield_11701'] = global_min.strftime("%Y-%m-%d")
    if 'customfield_11702' not in task_json:
        pass
        if sheet[schema['columns']['Estimate [h]'] + str(index)].value is not None:
            task_json['customfield_11702'] = (global_min + datetime.timedelta(hours = float(sheet[schema['columns']['Estimate [h]'] + str(index)].value)*3)).strftime("%Y-%m-%d")
        else:
            task_json['customfield_11702'] = (global_min + datetime.timedelta(days = 1)).strftime("%Y-%m-%d")

    ## planned start and finish time
    if 'customfield_11701' in task_json:
        task_json['customfield_11724'] = task_json['customfield_11701']
    if 'customfield_11702' in task_json:
        task_json['customfield_11725'] = task_json['customfield_11702']
        
    ## duration
    if sheet[schema['columns']['Duration [h]'] + str(index)].value is not None:
        task_json['customfield_10004'] = float(sheet[schema['columns']['Duration [h]'] + str(index)].value)/4
    else:
        if sheet[schema['columns']['Estimate [h]'] + str(index)].value is not None:
            task_json['customfield_10004'] = float(sheet[schema['columns']['Estimate [h]'] + str(index)].value)/4
    
    return task_json



def schema_FE(sheet, schema, index, task_json):
    if sheet[schema['columns']['Start time'] + str(index)].value is not None:
        task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
    if sheet[schema['columns']['Finish time'] + str(index)].value is not None:
        task_json['customfield_11702'] = datetime_to_date_converter(sheet[schema['columns']['Finish time'] + str(index)].value)

    if sheet[schema['columns']['Start time'] + str(index)].value is not None:
        task_json['customfield_11724'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
    if sheet[schema['columns']['Finish time'] + str(index)].value is not None:
        task_json['customfield_11725'] = datetime_to_date_converter(sheet[schema['columns']['Finish time'] + str(index)].value)

    if sheet[schema['columns']['Estimate [h]'] + str(index)].value is not None:
        task_json['customfield_10004'] = float(sheet[schema['columns']['Estimate [h]'] + str(index)].value)/4
        
    return task_json


def create_or_update_issue(sheets, sheet, schema, index, project, type, label, schema_type, parent = None, jira = None):

    print("current sheet: ", sheet, "    current row: ", index)

    ### form basic fields for create or update
    task_json = {
        'summary': sheet[schema['columns']['Name'] + str(index)].value,
        'description': sheet[schema['columns']['Description'] + str(index)].value,
        'components': [{"name": schema_type}]
    }

    if schema_type == 'FE':
        task_json = schema_FE(sheet, schema, index, task_json)
    if schema_type == 'BE' or schema_type == 'QA':
        task_json = schema_BE(sheets, sheet, schema, index, task_json)

    ## create issue
    if sheet[schema['columns']['Jira ID'] + str(index)].value is None: 

        task_json['project'] = {'key': project}
        task_json['issuetype'] = {'name': type}
        if parent is not None:
            task_json['parent'] = {'key': parent}

        if jira is not None:
            issue = jira.issue_create(task_json)
            # print(issue)
            sheet[schema['columns']['Jira ID'] + str(index)].value = issue['key']
            sheet[schema['columns']['Jira ID'] + str(index)].hyperlink = jira.url + '/browse/' + issue['key']
            sheet[schema['columns']['Jira ID'] + str(index)].font = Font(underline='single', color='0000FF')

    ## update basic fields     
    else:        
        if jira is not None:    
            jira.issue_update(sheet[schema['columns']['Jira ID'] + str(index)].value, task_json)
        # remove_all_links(jira_info, sheet[schema['columns']['Jira ID'] + str(index)].value)

    # print(task_json)

    ### section for edit issue

    ## add dependencies
    if sheet[schema['columns']['Depends on'] + str(index)].value is not None:
        if sheet[schema['columns']['Depends on'] + str(index)].value != '':
            # print(parse_dependencies(sheet[schema['columns']['Depends on'] + str(index)].value))
            for dependent_task_wbs in parse_dependencies(sheet[schema['columns']['Depends on'] + str(index)].value):
                dependent_jira_id = get_jira_key_for_wbs(schema, sheets, dependent_task_wbs.strip())
                if dependent_jira_id is not None:
                    # print(dependent_jira_id)
                    fields = {
                        "issuelinks": [
                            {
                                "add": {
                                    "type": {
                                        "name": "Gantt End to Start",
                                        "inward": "has to be done after",
                                        "outward": "has to be done before"
                                    },
                                    "inwardIssue": {
                                        "key": dependent_jira_id
                                    }
                                }
                            }
                        ]
                    }
                    # add dependency
                    if jira is not None:
                        jira.edit_issue(issue_id_or_key=sheet[schema['columns']['Jira ID'] + str(index)].value, fields=fields, notify_users=False)

    ## add labels
    labels = [{"add": label}]
    if sheet[schema['columns']['Category'] + str(index)].value is not None:
        labels = labels + [{"add": (str(sheet[schema['columns']['Category'] + str(index)].value)).strip().replace(' ', '_')}]

    if jira is not None:
        jira.edit_issue(issue_id_or_key=sheet[schema['columns']['Jira ID'] + str(index)].value, fields={"labels": labels}, notify_users=False)



def create_or_update_issues(jira, sheets, schema, project, label, schema_type):
    for sheet in sheets:
        print("current sheet: ", sheet)
        i = schema['data_offset_row']
        none_count = 0
        while none_count < 50:
            if sheet[schema['columns']['WBS'] + str(i)].value is not None:
                wbs = str(sheet[schema['columns']['WBS'] + str(i)].value)
                if not wbs_regex_check(wbs):
                    print('Problem in WBS number structure: ', wbs, ' in sheet: ', sheet)
                    i = i + 1
                    continue

                if sheet[schema['columns']['Name'] + str(i)].value is None or sheet[schema['columns']['Name'] + str(i)].value == '':
                    print('Task do not have any name, wbs: ', wbs, ' in sheet: ', sheet)
                    i = i + 1
                    continue

                create_or_update_issue(jira=jira, sheets=sheets, sheet=sheet, schema=schema, index=i, project=project, type='Task', label=label, schema_type=schema_type)

                none_count = 0
            else:
                none_count = none_count + 1
            i = i + 1
            # if i > 5:
            #     return


In [45]:
from openpyxl import load_workbook
from openpyxl.styles import Font


filename = 'P2 Unit Test Estimations.xlsx'
label = 'P2v3frontend'
schema_type = 'FE'


exclude_tab = ["PMSummary"]

workbook = load_workbook(filename=filename, data_only=True)

sheets_to_analyze = []

for sheet in workbook.sheetnames:
    if sheet not in exclude_tab:
        sheets_to_analyze.append(workbook[sheet])

print(workbook.sheetnames)
print(sheets_to_analyze)

['PMSummary', 'Components', 'Services', 'Utilities', 'Pipes', 'Directives', 'Misc.', 'Auth0']
[<Worksheet "Components">, <Worksheet "Services">, <Worksheet "Utilities">, <Worksheet "Pipes">, <Worksheet "Directives">, <Worksheet "Misc.">, <Worksheet "Auth0">]


In [45]:
from openpyxl import load_workbook
from openpyxl.styles import Font


# filename = 'O2 Grouper Pipeline - Task List.xlsx'
# label = 'P2_pipelines'
# schema_type = 'BE'

filename = 'P2 v3 Replatforming Tasks List.xlsx'
label = 'P2_v3'
schema_type = 'BE'

# filename = 'P2 v3 Authorization_DS manager - list of tasks.xlsx'
# label = 'Authorization'
# schema_type = 'BE'

# filename = 'P2 testing tasks.xlsx'
# label = 'Tests'
# schema_type = 'QA'


workbook = load_workbook(filename=filename, data_only=True)

sheets_to_analyze = [workbook['Tasks']]
  
print(workbook.sheetnames)
print(sheets_to_analyze)

['Description', 'Tasks', 'Bugs', 'Planned availability']
[<Worksheet "Tasks">]


In [46]:
schema = get_schema_from_sheet(sheets_to_analyze, schema_type)
create_or_update_issues(jira=None, sheets=sheets_to_analyze, schema=schema, project='PP', label=label, schema_type=schema_type)
create_or_update_issues(jira=None, sheets=sheets_to_analyze, schema=schema, project='PP', label=label, schema_type=schema_type)
create_or_update_issues(jira=None, sheets=sheets_to_analyze, schema=schema, project='PP', label=label, schema_type=schema_type)
create_or_update_issues(jira=None, sheets=sheets_to_analyze, schema=schema, project='PP', label=label, schema_type=schema_type)
create_or_update_issues(jira=None, sheets=sheets_to_analyze, schema=schema, project='PP', label=label, schema_type=schema_type)
create_or_update_issues(jira=jira, sheets=sheets_to_analyze, schema=schema, project='PP', label=label, schema_type=schema_type)
print("!!! FINISHED !!!")

current sheet:  <Worksheet "Tasks">
current sheet:  <Worksheet "Tasks">     current row:  3
current sheet:  <Worksheet "Tasks">     current row:  4
current sheet:  <Worksheet "Tasks">     current row:  5
current sheet:  <Worksheet "Tasks">     current row:  6
current sheet:  <Worksheet "Tasks">     current row:  7
current sheet:  <Worksheet "Tasks">     current row:  8
current sheet:  <Worksheet "Tasks">     current row:  9
current sheet:  <Worksheet "Tasks">     current row:  10
current sheet:  <Worksheet "Tasks">     current row:  11
current sheet:  <Worksheet "Tasks">     current row:  12
current sheet:  <Worksheet "Tasks">     current row:  13
current sheet:  <Worksheet "Tasks">     current row:  14
current sheet:  <Worksheet "Tasks">     current row:  15
current sheet:  <Worksheet "Tasks">     current row:  16
current sheet:  <Worksheet "Tasks">     current row:  17
current sheet:  <Worksheet "Tasks">     current row:  18
current sheet:  <Worksheet "Tasks">     current row:  19
cu

In [35]:
workbook.save(filename=filename)
workbook.close()

In [ ]:
map = {
    "A": "wbs",
    "B": "name",
    "C": "description",
    "D": "dependency",
    "E": "estimated_development_time_designer",
    "F": "estimated_development_time_reviewer",
    "G": "estimated_development_time_pm",
    "H": "assigned_developer",
    "I": "estimated_testing_time_tester",
    "J": "estimated_testing_time_pm",
    "K": "assigned_tester",
    "L": "actual_development_time_start",
    "M": "actual_development_time_finish",
    "N": "actual_development_time_duration",
    "O": "commit_statistics_insertions",
    "P": "commit_statistics_deletions",
    "Q": "code_documentation_time",
    "R": "jira_id",
    "S": "commit_id",
    "T": "sprint",
    "U": "comment"
}

tasks_to_json = []

i = schema['data_offset_row']
while sheet[schema['columns']['WBS'] + str(i)].value is not None:
    if len(str(sheet[schema['columns']['Name'] + str(i)].value)) != 0:
        task = {}
        for column in map:
            task[map[column]] = str(sheet[column + str(i)].value)
        tasks_to_json.append(task)
    i = i + 1 

print(tasks_to_json)

In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import Font

workbook = load_workbook(filename='summary.xlsx')
sheet = workbook['Sheet1']

i = 1
while sheet['A' + str(i)].value is not None:

    print(sheet['A' + str(i)].value)
    print(sheet['c'+str(i)].value + sheet['d'+str(i)].value + sheet['e'+str(i)].value)
    issue = jira.issue_create({
            'summary': 'Assign categories to ' + sheet['d'+str(i)].value + ' testcases',
            # 'description': sheet[schema['columns']['Description'] + str(index)].value,
            'project': {'key': 'PP'},
            'issuetype': {'name': 'Sub-task'},
            'parent': {'key': 'PP-162'},
            # 'Priority': 'priority',
            # 'Status': 'status',
            # 'Creator': 'creator',
            # 'Assignee': 'assignee',
        })

    i = i + 1

In [ ]:
print('February 22, 2021, 5:13 PM')

In [ ]:
from dateutil import parser

parser.parse(None)

In [ ]:
import datetime
parser.parse('2021-5-5') + datetime.timedelta(hours = 5.5)


In [22]:
## remove all links
import requests
from requests.auth import HTTPBasicAuth
import json

def remove_all_links(jira_info, issue_id):

    url_get = "https://tangramcare.atlassian.net/rest/api/2/issue/{0}?fields=issuelinks".format(issue_id)

    auth = HTTPBasicAuth(jira_info['username'], jira_info['api_token'])

    headers = {
    "Accept": "application/json",
    }

    response = requests.request(
    "GET",
    url_get,
    headers=headers,
    auth=auth
    )

    for i in response.json()['fields']['issuelinks']:

        url_delete = "https://tangramcare.atlassian.net/rest/api/2/issueLink/{0}".format(i['id'])
        print(url_delete, "   ", i['id'])
        response = requests.request(
            "DELETE",
            url_delete,
            auth=auth
        )



In [17]:
response

<Response [200]>